## Convert IA : keras to onnx file


In [3]:
#***************************************************************
# Endvironment : myconvert
#***************************************************************
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
import cv2
import onnx
import tensorflow as tf
import tf2onnx
import onnx 
import onnxruntime as rt
import numpy as np
import pandas

json_file = open('convautoencodermodel_10.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("convautoencodermodel_10.h5")
jearsey_layer_model = Model(inputs=loaded_model.input,outputs=loaded_model.get_layer("conv2d_15").output)

#****************************************************************
#     Binding of inputs : name is essential for binding in c++
#****************************************************************
input_signature = [tf.TensorSpec([1, 64, 64, 3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(jearsey_layer_model, input_signature, opset=13)
output_names = [n.name for n in onnx_model.graph.output]
onnx.save(onnx_model, "./model.onnx")

patches = []
patche = cv2.imread('visage.png')
patches.append(cv2.resize(patche, (64, 64)))

providers = ['CPUExecutionProvider']
m = rt.InferenceSession("./model.onnx", providers=providers)
onnx_pred = m.run(output_names, {"x": patches})

representation = jearsey_layer_model.predict(np.array(patches))
A = np.array(representation - onnx_pred[0])
print(np.max(A))


INFO:tensorflow:Assets written to: /home/eekmekci/projects/keras/convert/keras2cpp/new_model.pb/assets


INFO:tensorflow:Assets written to: /home/eekmekci/projects/keras/convert/keras2cpp/new_model.pb/assets
2025-04-18 12:10:32.980827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-18 12:10:32.980987: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-04-18 12:10:32.981071: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2025-04-18 12:10:32.981489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI

1/1 [==============================] - 0s 33ms/step
0.00030517578
